## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-30-08-46-30 +0000,bbc,Why Starmer wants to keep talking about Farage,https://www.bbc.com/news/articles/cge20pnx1rqo...
1,2025-09-30-08-46-08 +0000,nyt,Rescuers in Indonesia Race to Save Students Tr...,https://www.nytimes.com/2025/09/30/world/asia/...
2,2025-09-30-08-42-24 +0000,bbc,Former aide to far-right German politician jai...,https://www.bbc.com/news/articles/c99g52y7k1xo...
3,2025-09-30-08-34-27 +0000,bbc,Blair would help oversee Gaza transition under...,https://www.bbc.com/news/articles/cq5j989107lo...
4,2025-09-30-08-33-00 +0000,wsj,French Inflation Picks Up Pace as ECB Looks Li...,https://www.wsj.com/world/europe/french-inflat...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2295/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
28,trump,52
58,shutdown,15
122,new,15
40,government,14
25,gaza,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
44,2025-09-30-03-05-00 +0000,wsj,Top Democrats made their case to Trump for res...,https://www.wsj.com/politics/policy/government...,132
141,2025-09-29-21-14-29 +0000,nypost,Trump unveils 20-point Gaza cease-fire plan wi...,https://nypost.com/2025/09/29/us-news/trump-un...,132
114,2025-09-29-22-29-55 +0000,nypost,Ex-UK PM Tony Blair to help run Gaza under Tru...,https://nypost.com/2025/09/29/us-news/ex-uk-pm...,109
22,2025-09-30-06-50-20 +0000,nypost,Trump mocks Chuck Schumer and Hakeem Jeffries ...,https://nypost.com/2025/09/30/us-news/trump-mo...,104
153,2025-09-29-20-50-55 +0000,nyt,Read Each Point of Trump’s Plan for an Israel-...,https://www.nytimes.com/2025/09/29/world/middl...,101


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
44,132,2025-09-30-03-05-00 +0000,wsj,Top Democrats made their case to Trump for res...,https://www.wsj.com/politics/policy/government...
141,80,2025-09-29-21-14-29 +0000,nypost,Trump unveils 20-point Gaza cease-fire plan wi...,https://nypost.com/2025/09/29/us-news/trump-un...
178,52,2025-09-29-18-55-45 +0000,nypost,Frank founder Charlie Javice sobs in court as ...,https://nypost.com/2025/09/29/business/charlie...
88,44,2025-09-30-00-01-00 +0000,wsj,American Eagle Outfitters’ chief executive had...,https://www.wsj.com/business/retail/you-cant-r...
254,43,2025-09-29-14-37-00 +0000,wsj,"Eric Adams Ends Re-Election Bid, Reshaping Rac...",https://www.wsj.com/us-news/new-york-city-mayo...
214,41,2025-09-29-16-49-04 +0000,cbc,Police continue to search for motive in deadly...,https://www.cbc.ca/news/canada/windsor/deadly-...
22,37,2025-09-30-06-50-20 +0000,nypost,Trump mocks Chuck Schumer and Hakeem Jeffries ...,https://nypost.com/2025/09/30/us-news/trump-mo...
151,35,2025-09-29-20-53-00 +0000,wsj,The Trump administration said it is cracking d...,https://www.wsj.com/politics/national-security...
283,34,2025-09-29-12-05-00 +0000,wsj,Moldova’s Pro-Europe Party Claims Victory Over...,https://www.wsj.com/world/europe/moldovas-pro-...
240,34,2025-09-29-15-31-00 +0000,wsj,President Trump said films made outside the U....,https://www.wsj.com/politics/elections/trump-r...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
